# Settings

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/vidie-audie

In [ ]:
%cd /content/drive/MyDrive/vidie-audie

In [ ]:
# env loading
!pip install python-dotenv

from dotenv import load_dotenv
import os

dotenv_path = '/content/drive/MyDrive/vidie-audie/.env'

load_dotenv(dotenv_path)
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# whisper 모델 설치
!pip install git+https://github.com/openai/whisper.git -q

# Whisper 모델 로드 (Medium 사이즈)
import whisper
model = whisper.load_model("medium")

# Youtube MP3 추출에 필요한 라이브러리
!pip install -U yt-dlp

# Flask 서버에 필요한 라이브러리 설치 및 토큰 설정
!pip install flask flask-ngrok pyngrok flask-cors
!ngrok authtoken {ngrok_token}

# gpt api를 위한 라이브러리
!pip install openai

In [ ]:
!sudo apt-get update
!sudo apt-get install -y libmariadb-dev-compat libmariadb-dev

# mariadb 모듈
!pip install mariadb

In [ ]:
model.device

# Process code on Flask

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import yt_dlp
import whisper
import shutil
import os
import re
from flask_cors import CORS
import urllib.parse

import openai
import time

import requests
import mariadb
from datetime import datetime

# Settings에서 load한 env
KAKAO_API_HOST = "https://kapi.kakao.com"
KAKAO_REST_API_KEY = os.getenv('KAKAO_REST_API_KEY')


app = Flask(__name__)
CORS(app)

public_url = ngrok.connect(5000)
print("ngrok tunnel available at:", public_url)


# MariaDB 설정
MYSQL_DATABASE_USER = os.getenv('MYSQL_DATABASE_USER')
MYSQL_DATABASE_PASSWORD = os.getenv('MYSQL_DATABASE_PASSWORD')
MYSQL_DATABASE_DB = os.getenv('MYSQL_DATABASE_DB')
MYSQL_DATABASE_HOST = os.getenv('MYSQL_DATABASE_HOST')
MYSQL_DATABASE_PORT = os.getenv('MYSQL_DATABASE_PORT')

def get_db_connection():
    try:
        print("Connecting to database...")
        connection = mariadb.connect(
            user=MYSQL_DATABASE_USER,
            password=MYSQL_DATABASE_PASSWORD,
            host=MYSQL_DATABASE_HOST,
            database=MYSQL_DATABASE_DB,
            port=int(MYSQL_DATABASE_PORT)
        )
        print("Database connection successful")
        return connection
    except mariadb.Error as err:
        print("Database connection error:", err)
        if err.errno == 1045: # ACCESS_DENIED_ERROR
            print("Something is wrong with your user name or password")
        elif err.errno == 1049: # BAD_DB_ERROR
            print("Database does not exist")
        else:
            print(err)
        return None


@app.route('/kakaoLogin', methods=['POST'])
def kakao_login():
    try:
        print("Received request for /kakaoLogin")
        token = request.json.get('token')
        print(f"Token received: {token}")
        headers = {
            'Authorization': f'Bearer {token}',
            'Content-Type': 'application/x-www-form-urlencoded;charset=utf-8'
        }
        user_info_response = requests.get(f'{KAKAO_API_HOST}/v2/user/me', headers=headers)
        print("Kakao API response received")

        if user_info_response.status_code != 200:
            print("Failed to fetch user info from Kakao")
            return jsonify({'error': 'Failed to fetch user info from Kakao'}), user_info_response.status_code

        user_info = user_info_response.json()
        print(f"User info: {user_info}")

        user_id = user_info['id']
        nickname = user_info['properties']['nickname']
        connected_at = user_info['connected_at']

        # Convert connected_at to MySQL DATETIME format
        connected_at = datetime.strptime(connected_at, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')

        db_conn = get_db_connection()
        if db_conn is None:
            print('Database connection failed')
            return jsonify({'error': 'Failed to connect to database'}), 500

        cursor = db_conn.cursor()
        try:
            print("Inserting or updating user info in the database")
            cursor.execute("""
                INSERT INTO users (id, nickname, connected_at)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE nickname=%s, connected_at=%s
            """, (user_id, nickname, connected_at, nickname, connected_at))
            db_conn.commit()
            print("Database operation successful")
        except mariadb.Error as err:
            print(f"Error executing SQL: {err}")
            return jsonify({'error': str(err)}), 500
        finally:
            cursor.close()
            db_conn.close()

        return jsonify(user_info)
    except Exception as e:
        print(f"Unexpected error: {e}")
        return jsonify({'error': 'Internal Server Error'}), 500

@app.route('/user/<int:user_id>/add_record', methods=['POST'])
def add_record(user_id):
    try:
        data = request.json
        url = data.get('url')
        title = data.get('title')
        keyword = data.get('keyword')
        summary = data.get('summary')
        if not url or not title or not keyword or not summary:
            return jsonify({'error': 'More data is required'}), 400

        db_conn = get_db_connection()
        if db_conn is None:
            print('Database connection failed')
            return jsonify({'error': 'Failed to connect to database'}), 500

        cursor = db_conn.cursor()
        try:
            print(f"Inserting record for user {user_id}: URL={url}, Title={title}...")
            cursor.execute("""
                INSERT INTO user_records (user_id, title, summary, video_url, keyword)
                VALUES (%s, %s, %s, %s, %s)
            """, (user_id, title, summary, url, keyword))
            db_conn.commit()
            print("Record insertion successful")
        except mariadb.Error as err:
            print(f"Error executing SQL: {err}")
            return jsonify({'error': str(err)}), 500
        finally:
            cursor.close()
            db_conn.close()

        return jsonify({'status': 'success'}), 201
    except Exception as e:
        print(f"Unexpected error: {e}")
        return jsonify({'error': 'Internal Server Error'}), 500

@app.route('/user/<int:user_id>/records', methods=['GET'])
def get_records(user_id):
    db_conn = get_db_connection()
    if db_conn is None:
        print('Database connection failed')
        return jsonify({'error': 'Failed to connect to database'}), 500

    cursor = db_conn.cursor()
    try:
        cursor.execute("""
            SELECT id, video_url, title, created_at FROM user_records WHERE user_id = %s
        """, (user_id,))
        records = cursor.fetchall()

        # Process the video_url to extract the part after "watch?v="
        processed_records = []
        for record in records:
            video_url = record[1]
            video_id = video_url.split('watch?v=')[-1] if 'watch?v=' in video_url else video_url
            processed_records.append((record[0], video_id, record[2], record[3]))
    except mariadb.Error as err:
        print(f"Error executing SQL: {err}")
        return jsonify({'error': str(err)}), 500
    finally:
        cursor.close()
        db_conn.close()

    return jsonify(processed_records)

@app.route('/video_title/<path:video_id>', methods=['OPTIONS'])
def options_video_title(video_id):
    # 필요한 CORS 헤더를 설정하여 OPTIONS 요청에 대응
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET, POST, OPTIONS',
        'Access-Control-Allow-Headers': 'Content-Type, Authorization, ngrok-skip-browser-warning'
    }
    return '', 200, headers

@app.route('/video_title/<path:video_id>', methods=['GET'])
def get_video_title(video_id):
    video_id = urllib.parse.unquote(video_id)
    print(f"Received request for video title with ID: {video_id}")
    db_conn = get_db_connection()
    if db_conn is None:
        print("Database connection failed")
        return jsonify({'error': 'Failed to connect to database'}), 500

    cursor = db_conn.cursor()
    print(cursor)
    title = "Title not found"
    try:
        cursor.execute("""
            SELECT title FROM user_records WHERE video_url = %s
        """, (video_id,))
        result = cursor.fetchone()
        print(f"Query result: {result}")
        if result:
            title = result[0]
        # Read all results to avoid "Unread result found" error
        cursor.fetchall()
    except mariadb.Error as err:
        print(f"Error executing SQL: {err}")
        return jsonify({'error': str(err)}), 500
    finally:
        cursor.close()
        db_conn.close()
    response = jsonify({'title': title})
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type, Authorization,ngrok-skip-browser-warning')

    return response

@app.route('/user/<int:user_id>/top_keywords', methods=['GET'])
def get_top_keywords(user_id):
    db_conn = get_db_connection()
    if db_conn is None:
        print('데이터베이스 연결 실패')
        return jsonify({'error': '데이터베이스 연결 실패'}), 500

    cursor = db_conn.cursor()
    try:
        cursor.execute("""
            SELECT keyword, COUNT(*) as keyword_count
            FROM user_records
            WHERE user_id = %s
            GROUP BY keyword
            ORDER BY keyword_count DESC
            LIMIT 2
        """, (user_id,))
        results = cursor.fetchall()
        print("최다 키워드 조회 성공")
    except mariadb.Error as err:
        print(f"SQL 실행 오류: {err}")
        return jsonify({'error': str(err)}), 500
    finally:
        cursor.close()
        db_conn.close()

    if results:
        top_keywords = [{'keyword': result[0], 'count': result[1]} for result in results]
        return jsonify(top_keywords)
    else:
        return jsonify({'error': 'No keywords found'}), 404

@app.route('/video/<path:video_id>/keywords', methods=['GET'])
def get_keywords(video_id):
    db_conn = get_db_connection()
    if db_conn is None:
        print('데이터베이스 연결 실패')
        return jsonify({'error': '데이터베이스 연결 실패'}), 500

    cursor = db_conn.cursor()
    try:
        cursor.execute("""
            SELECT keyword FROM user_records WHERE video_url = %s
        """, (video_id,))
        keywords = cursor.fetchall()
        print("키워드 조회 성공")
    except mariadb.Error as err:
        print(f"SQL 실행 오류: {err}")
        return jsonify({'error': str(err)}), 500
    finally:
        cursor.close()
        db_conn.close()

    return jsonify([keyword[0] for keyword in keywords])

#요약 내용 가져요기
@app.route('/summary/<path:video_id>', methods=['GET'])
def get_latest_summary(video_id):
    db_conn = get_db_connection()
    if db_conn is None:
        print('데이터베이스 연결 실패')
        return jsonify({'error': '데이터베이스 연결 실패'}), 500

    cursor = db_conn.cursor()
    try:
        cursor.execute("""
            SELECT summary FROM user_records
            WHERE video_url = %s
            ORDER BY created_at DESC
            LIMIT 1
        """, (video_id,))
        summary = cursor.fetchone()
        if summary:
            summary = summary[0]
            print("최근 요약 조회 성공")
        else:
            summary = "Summary not found"
            print("요약을 찾을 수 없습니다.")
    except mariadb.Error as err:
        print(f"SQL 실행 오류: {err}")
        return jsonify({'error': str(err)}), 500
    finally:
        cursor.close()
        db_conn.close()

    return jsonify({'summary': summary})

# 추출->변환->요약 프로세스
def sanitize_filename(filename):
    """파일 이름에서 허용되지 않는 문자를 제거합니다."""
    return re.sub(r'[\\/*?:"<>|\']', "", filename)

def download_video_as_mp3(url, custom_name):
    safe_custom_name = sanitize_filename(custom_name)
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': f"/content/drive/My Drive/vidie-audie/contents/{safe_custom_name}.%(ext)s",
        'windowsfilenames': False  # 윈도우 파일 이름 규칙을 따르지 않도록 설정 #다루고 있는 safe_custom_name과 실제 파일명이 달라질 수 있는 잠재적 가능성 방지
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
        return safe_custom_name


client = openai.OpenAI(
    api_key=openai_api_key
)

SUMMARY_PROMPT = """입력되는 텍스트를 500자 내외로 요약해주고, 해당 텍스트의 핵심 키워드 단어를 하나 골라서 요약문과 같이 제공해줘
너가 제공해줘야 할 출력 형태는 '키워드 단어 & 500자 내외의 요약문'이야. 출력 형태를 반드시 지켜"""

def read_text_file(file_path) :
  with open(file_path, 'r', encoding='utf-8') as file:
    return file.read()

def summarize_text(text) :
  max_retries = 5
  for i in range(max_retries) :
      try :
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role" : "system", "content" : SUMMARY_PROMPT},
                {"role": "user", "content" : text}
            ],
            temperature=0.5 # 0.0: 결정적, 1.0: 창의적
        )
        message_content = response.choices[0].message.content.strip()
        if '&' in message_content :
          if len(message_content.split(' & ')[0]) < 10:
            return message_content
        else :
          print("응답이 예상한 것과 다르게 나왔습니다. 다시 시도합니다.")

      except openai.RateLimitError:
        print(f"RateLimitError 발생 {2 ** i}초 뒤 재호출하겠습니다.")
        time.sleep(2 ** i)
      except openai.APIError as e :
        print(f"OpenAIError 발생 : {str(e)}")
        raise e

  raise Exception("error : 최대 재시도 횟수 초과")



@app.route("/MainPage", methods=["POST"])
def process() :
  data = request.json
  down_url = data['url']
  custom_name = data['title']

  title = download_video_as_mp3(down_url, custom_name)
  print("mp3 다운로드 완료!")
  mp3_file_path = f"/content/drive/My Drive/vidie-audie/contents/{title}.mp3"

  result = model.transcribe(mp3_file_path)

  txt_file_path = f"/content/drive/My Drive/vidie-audie/contents/{title}.txt"

  with open(txt_file_path, 'w', encoding='utf-8') as file:
      file.write(result['text'])
  print(f"텍스트 파일 저장 완료! 파일 경로: {txt_file_path}")

  input_file_path = txt_file_path
  text = read_text_file(input_file_path)
  text_output = summarize_text(text).split(' & ')

  keyword = text_output[0]
  summary = text_output[1]

  print('response complete')
  return jsonify({"message": "Processing complete", "keyword": keyword, "summary": summary})

app.run(port=5000)